In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import (
    run_full_analysis, 
    print_continuous_status, 
    download_analysis,
    DEFAULT_RUN_CONFIG,
)

In [ ]:
DEFAULT_RUN_CONFIG

In [ ]:
ISO3_LIST = ["AUS", "BOL", "FRA"]
ANALYSIS = "main"

N_CORES = DEFAULT_RUN_CONFIG['N_CORES']

N_CHAINS = DEFAULT_RUN_CONFIG['N_CHAINS']
N_OPTI_SEARCHES = DEFAULT_RUN_CONFIG['N_OPTI_SEARCHES']
OPTI_BUDGET = DEFAULT_RUN_CONFIG['OPTI_BUDGET']

METROPOLIS_TUNE = DEFAULT_RUN_CONFIG['METROPOLIS_TUNE']
METROPOLIS_DRAWS = DEFAULT_RUN_CONFIG['METROPOLIS_DRAWS']
METROPOLIS_METHOD = DEFAULT_RUN_CONFIG['METROPOLIS_METHOD']

FULL_RUNS_SAMPLES = DEFAULT_RUN_CONFIG['FULL_RUNS_SAMPLES']
BURN_IN = DEFAULT_RUN_CONFIG['BURN_IN']

In [ ]:
assert (METROPOLIS_DRAWS - BURN_IN) * N_CHAINS >= FULL_RUNS_SAMPLES
assert METROPOLIS_METHOD in ("DEMetropolis", "DEMetropolisZ")

In [ ]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        n_cores=N_CORES,
        analysis=analysis, 
        opti_params={'n_searches': N_OPTI_SEARCHES, 'search_iterations': OPTI_BUDGET},
        mcmc_params={'draws': METROPOLIS_DRAWS, 'tune': METROPOLIS_TUNE, 'chains': N_CHAINS, 'method': METROPOLIS_METHOD},
        full_run_params={'samples': FULL_RUNS_SAMPLES, 'burn_in': BURN_IN},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")

In [ ]:
analysis_title = "test_multirun"
config_str = f"_{ANALYSIS}_LHS{N_OPTI_SEARCHES}_opt{OPTI_BUDGET}_mc{METROPOLIS_TUNE}n{METROPOLIS_DRAWS}"
run_paths = [springboard.launch.get_autumn_project_run_path("school_project", iso3, analysis_title + config_str) for iso3 in ISO3_LIST]
run_paths

In [ ]:
task_dict = {}
for i, iso3 in enumerate(ISO3_LIST):
    # Use these targets, but keep the other kwargs the same
    task_kwargs = {
        "iso3": iso3,
        "analysis": ANALYSIS
    }
    task_dict[run_paths[i]] = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
springboard.launch.launch_synced_multiple_autumn_task?

In [ ]:
runners = springboard.launch.launch_synced_multiple_autumn_task(task_dict, mspec, run_group="2023-09-11T1023-test_multirun_main_LHS16_opt10000_mc5000n30000")


In [ ]:
for run_path, runner in runners.items():
    print(runner.s3.get_status())
    # print(runner.top("%MEM"))

In [ ]:
run_paths

In [ ]:
for run_path in run_paths:
    download_analysis(run_path)